# **CIVIL AVIATION - DOMESTIC AIRLINES ANALYTICS - IND**

**Let's try to predict the the data for 2020, 2021, 2022, iff COVID hadn't happened  and compare with the actual data**

**1. IMPORT LIBRARIES**

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import h5py
from datetime import datetime

import requests as re
import glob

import os
import warnings
warnings.filterwarnings('ignore')

**2. READ DATA & DO INITAL CHECK's AND MODS**

In [2]:
file = glob.glob("file_path\*")

Read 1 file

In [3]:
xl = pd.ExcelFile(file[0])

Each carrier-data file has 4 tables
1. row# : 01 - 15, agg_line# : 16
2. row# : 20 - 34, agg_line# : 35
3. row# : 39 - 53, agg_line# : 54
4. row# : 58 - 72, agg_line# : 73

Let's concentrate on 1st table only.

In [4]:
df = xl.parse(0, nrows=14)
df.head()

,2021,Unnamed: 1,Monthly Traffic And Operating Statistics (Provisional)\n ( Scheduled Domestic Services),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,AIR ASIA,Unnamed: 15,Unnamed: 16
0,MONTH,AIRCRAFT FLOWN,NaN,NaN,PASSENGERS CARRIED\n(IN NUMBER),PASSENGER KMS.PERFORMED\n(IN THOUSAND),AVAILABLE SEAT KILOMETRE\n(IN THOUSAND),PAX.LOAD FACTOR#\n(IN %),CARGO CARRIED,NaN,NaN,TONNE KILOMETRE PERFORMED,NaN,NaN,NaN,AVAILABLE TONNE KILOMETRE\n(IN THOUSAND),WEIGHT LOAD FACTOR##\n(IN %)
1,NaN,DEPARTURES\n(IN NUMBER),HOURS\n(IN NUMBER),KILOMETRE\n(IN THOUSAND),NaN,NaN,NaN,NaN,FREIGHT\n(IN TONNE),MAIL\n(IN TONNE),TOTAL\n(IN TONNE),PASSENGER\n(IN THOUSAND),FREIGHT\n(IN THOUSAND),MAIL\n(IN THOUSAND),TOTAL\n(IN THOUSAND),NaN,NaN
2,JAN,4495,8590.4,4621.139,531578,558871.307,836264.022,66.829529,3175.7262,230.1885,3405.9147,37494.472877,4067.437374,269.242257,41831.152508,92422.78,45.260652
3,FEB,4340,8239.033333,4417,520603,542276.763,798746.808,67.890946,3410.6377,211.8922,3622.5299,39555.584103,4264.010976,251.988624,44071.583703,88341.12,49.887961
4,MAR,4727,9023.3,4855.674,541512,572123.104,878405.634,65.131994,3662.2624,211.7535,3874.0159,38524.441947,4577.131657,280.847992,43382.421596,97113.48,44.671884


Extract the year and airlines name.

In [5]:
carrier_map = {
    'Air Asia' : ['air asia','airasia']
    , 'Star Air' : ['star air']
    , 'Air Carnival' : ['air carnival', 'aircarnival']
    , 'air costa' : ['air costa', 'aircosta']
    , 'Air Deccan' : ['air deccan', 'airdeccan']
    , 'Air Heritage' : ['airheritage', 'air heritage']
    , 'Air India Express' : ['air india express', 'airindiaexpress']
    , 'Air India' : ['air india', 'airindia']
    , 'air pegasus' : ['air pegasus', 'airpegasus']
    , 'Air Taxi' : ['air taxi']
    , 'Alliance Air' : ['alliance air', 'alliance']
    , 'Blue Dart' : ['bluedart']
    , 'Deccan Air' : ['deccan air', 'deccanair']
    , 'FlyBig' : ['flybig']
    , 'Go Air' : ['go air', 'goair']
    , 'IndiGo' : ['indigo']
    , 'Jet Airways' : ['jet airways', 'jetairways']
    , 'JetLite' : ['jetlite']
    , 'Pawan Hans' : ['pawanhans', 'pawan hans']
    , 'QUICKJET' : ['quickjetcargo']
    , 'SpiceJet' : ['spicejet']
    , 'Star Air' : ['star air', 'starair']
    , 'TruJet' : ['trujet']
    , 'vistara' : ['vistara ']
    , 'zoomair' : ['zoomair']
}

In [6]:
yr_map = {
    2022 : '22'
    , 2021 : '21'
    , 2020 : '20'
    , 2019 : '19'
    , 2018 : '18'
    , 2017 : '17'
    , 2016 : '16'
    , 2015 : '15'
    , 2014 : '14'
    , 2013 : '13'
    , 2012 : '12'
    , 2011 : '11'
    , 2010 : '10'
    , 2009 : '09'
}

In [7]:
file_name = file[0].split('\\')[-1].split('.')[0]
file_name

'Air Asia21'

In [8]:
carrier_name = np.nan
carrier_yr = np.nan

for k, val in carrier_map.items():
    for v in val:
        if pd.isna(carrier_name):
            if file_name[:-2].lower() in v:
                carrier_name = k
                break

    if pd.notna(carrier_name):
        break

print(f"Airlines Name : {carrier_name}")
        
for k, val in yr_map.items():
    if file_name[-2:] in val:
        carrier_yr = k
        break
    
print(f"Airlines Year : {carrier_yr}")

Airlines Name : Air Asia
Airlines Year : 2021


Create a dict for renaming the columns

In [9]:
col_names = {
    df.columns[0] : 'month(s)'
    , df.columns[1] : 'dep(in_num)'
    , df.columns[2] : 'hours(in_num)'
    , df.columns[3] : 'km(in_thousand)'
    , df.columns[4] : 'pax_carried(in_num)'
    , df.columns[5] : 'pax_km_pfd(in_thousand)'
    , df.columns[6] : 'avail_seat_km(in_thousand)'
    , df.columns[7] : 'pax_load_factor(in_percentage)'
    , df.columns[8] : 'freight(in_tonne)'
    , df.columns[9] : 'mail(in_tonne)'
    , df.columns[10] : 'total_cargo(in_tonne)'
    , df.columns[11] : 'pax_tonne_km_pfd(in_tonne)'
    , df.columns[12] : 'freight_tonne_km_pfd(in_tonne)'
    , df.columns[13] : 'mail_tonne_km_pfd(in_tonne)'
    , df.columns[14] : 'total_tonne_km_pfd(in_tonne)'
    , df.columns[15] : 'avl_tonne_km_pfd(in_tonne)'
    , df.columns[16] : 'weight_load_factor(in_percentage)'
}

| column_name | definition |
| --- | --- |
| dep | Count of Departures. |
| hours | Total hours flown. |
| km | Total distance flown. |
| pax_carried | Total number of passengers carried. |
| pax_km_pfd | _Passenger kilometers performed_ - A passenger-kilometre is performed when a passenger is carried one kilometre. Passenger-kilometres equal the sum of the products obtained by multiplying the number of revenue passengers carried on each flight stage by the stage distance. The resultant figure is equal to the number of kilometres travelled by all passengers. |
| avail_seat_km | _Available Seat Kilometers (ASK)_ - captures the total flight passenger capacity of an airline in kilometers. It is obtained by multiplying the total number of seats available for scheduled passengers and the total number of kilometers in which those seats were flown. <br> A seat-kilometer is available when a seat that is available for carrying a passenger is flown one kilometer. Seats that are not usable for various reasons are excluded. |
| pax_load_factor | _Passenger Load Factor (PLF) or Load Factor (LF)_ - is an airline industry metric that measures how much of an airline’s passenger carrying capacity is used. Not to be confused with aeronautic load factor, PLF only measures capacity utilization. <br> $ \frac{passenger-kms-performed}{available-seat-kms} $ |
| freight | weight of cargo carried. |
| mail | weight of mail carried. |
| total_cargo | total weight of cargo and mail carried. |
| pax_tonne_km_pfd | _Passenger Tonne Kilometers Performed_ - The result obtained by multiplying the passenger kilometres flown by the weight of each of the passengers including both free and excess baggage. <br> _Each air transport operator can use its own internal passenger weights or the standard 90kgs (baggage included)._ |

Rename and modify columns.

In [10]:
df = df.rename(columns=col_names).drop([0, 1])
df['pax_load_factor(in_percentage)'] = df['pax_load_factor(in_percentage)'].apply(lambda x: 0 if x == ' ' else x)
df['weight_load_factor(in_percentage)'] = df['weight_load_factor(in_percentage)'].apply(lambda x: 0 if x == ' ' else x)
df.fillna(0, inplace=True)
df.reset_index(inplace=True, drop=True)

Lets check the columns data type(s).

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   month(s)                           12 non-null     object 
 1   dep(in_num)                        12 non-null     int64  
 2   hours(in_num)                      12 non-null     float64
 3   km(in_thousand)                    12 non-null     float64
 4   pax_carried(in_num)                12 non-null     int64  
 5   pax_km_pfd(in_thousand)            12 non-null     float64
 6   avail_seat_km(in_thousand)         12 non-null     float64
 7   pax_load_factor(in_percentage)     12 non-null     float64
 8   freight(in_tonne)                  12 non-null     float64
 9   mail(in_tonne)                     12 non-null     float64
 10  total_cargo(in_tonne)              12 non-null     float64
 11  pax_tonne_km_pfd(in_tonne)         12 non-null     float64
 

Format columns data types

In [12]:
for col in df.columns:
    if col == 'month(s)':
        continue
    
    df[col] = df[col].astype('float64')

df['year'] = int(carrier_yr)
df['airlines'] = carrier_name

In [13]:
df.head()

,month(s),dep(in_num),hours(in_num),km(in_thousand),pax_carried(in_num),pax_km_pfd(in_thousand),avail_seat_km(in_thousand),pax_load_factor(in_percentage),freight(in_tonne),mail(in_tonne),total_cargo(in_tonne),pax_tonne_km_pfd(in_tonne),freight_tonne_km_pfd(in_tonne),mail_tonne_km_pfd(in_tonne),total_tonne_km_pfd(in_tonne),avl_tonne_km_pfd(in_tonne),weight_load_factor(in_percentage),year,airlines
0,JAN,4495.0,8590.400000,4621.139,531578.0,558871.307,836264.022,66.829529,3175.7262,230.1885,3405.9147,37494.472877,4067.437374,269.242257,41831.152508,92422.78,45.260652,2021,Air Asia
1,FEB,4340.0,8239.033333,4417.000,520603.0,542276.763,798746.808,67.890946,3410.6377,211.8922,3622.5299,39555.584103,4264.010976,251.988624,44071.583703,88341.12,49.887961,2021,Air Asia
2,MAR,4727.0,9023.300000,4855.674,541512.0,572123.104,878405.634,65.131994,3662.2624,211.7535,3874.0159,38524.441947,4577.131657,280.847992,43382.421596,97113.48,44.671884,2021,Air Asia
3,APR,3195.0,6341.383333,3481.586,354878.0,403399.335,630122.874,64.019154,2487.1216,169.2380,2656.3596,27963.798660,3332.789591,208.044115,31504.632366,69631.72,45.244656,2021,Air Asia
4,MAY,809.0,1562.150000,867.183,63532.0,70007.636,157745.280,44.380178,1344.1482,57.5075,1401.6557,4039.994933,1744.005877,114.999778,5899.000588,17343.66,34.012432,2021,Air Asia


**3. READ ALL DATA & MODIFIY**
<br>Now lets replicate the above steps for all the files.

In [14]:
temp_df = pd.DataFrame()

In [15]:
def get_airlines_name_yr(file_name):
    carrier_name = np.nan
    carrier_yr = np.nan
    file_name = file_name.split(os.sep)[-1].split('.')[0]

    for k, val in carrier_map.items():
        for v in val:
            if pd.isna(carrier_name):
                if file_name[:-2].lower() == v:
                    carrier_name = k
                    break

        if pd.notna(carrier_name):
            break

    for k, val in yr_map.items():
        if file_name[-2:] == val:
            carrier_yr = k
            break
            
    return carrier_name, carrier_yr

In [16]:
for f in file:
    if 'dom' in f.lower():
        continue
    elif 'total' in f.lower():
        continue
    else:
        xl = pd.ExcelFile(f)
        df = xl.parse(0, nrows=14)

        col_names = {
            df.columns[0] : 'month(s)'
            , df.columns[1] : 'dep(in_num)'
            , df.columns[2] : 'hours(in_num)'
            , df.columns[3] : 'km(in_thousand)'
            , df.columns[4] : 'pax_carried(in_num)'
            , df.columns[5] : 'pax_km_pfd(in_thousand)'
            , df.columns[6] : 'avail_seat_km(in_thousand)'
            , df.columns[7] : 'pax_load_factor(in_percentage)'
            , df.columns[8] : 'freight(in_tonne)'
            , df.columns[9] : 'mail(in_tonne)'
            , df.columns[10] : 'total_cargo(in_tonne)'
            , df.columns[11] : 'pax_tonne_km_pfd(in_tonne)'
            , df.columns[12] : 'freight_tonne_km_pfd(in_tonne)'
            , df.columns[13] : 'mail_tonne_km_pfd(in_tonne)'
            , df.columns[14] : 'total_tonne_km_pfd(in_tonne)'
            , df.columns[15] : 'avl_tonne_km_pfd(in_tonne)'
            , df.columns[16] : 'weight_load_factor(in_percentage)'
        }

        df = df.rename(columns=col_names).drop([0, 1])
        
        for col in df.columns:
            df[col] = df[col].apply(lambda x: 0 if x == ' ' else x)
            df[col] = df[col].replace({'NR':np.nan, '-':np.nan})
        
        df.fillna(0, inplace=True)
        df.reset_index(inplace=True, drop=True)

        for col in df.columns:
            if col == 'month(s)':
                continue
            
            df[col] = df[col].astype('float64')
        
        df['airlines'],df['year'] = get_airlines_name_yr(f)

        temp_df = pd.concat([temp_df, df], ignore_index=True)

In [17]:
df = temp_df.iloc[:, :-2]

In [18]:
# %reset -f